In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
import tensorflow as tf

In [2]:
f=open('Data.pickle','rb')
Data=open('Data.pickle','rb')
train_dataset=pickle.load(Data)['train_dataset']
Data.seek(0)
train_labels=pickle.load(Data)['train_labels']
Data.seek(0)
test_dataset=pickle.load(Data)['test_dataset']
Data.seek(0)
test_labels=pickle.load(Data)['test_labels']
Data.close()

In [3]:
print(train_dataset.shape)
print(test_dataset.shape)


(50000, 28, 28)
(900, 28, 28)


In [4]:
train_dataset=train_dataset.reshape(train_dataset.shape[0],-1).T
train_labels=train_labels.reshape(train_labels.shape[0],-1).T
test_dataset=test_dataset.reshape(test_dataset.shape[0],-1).T
test_labels=test_labels.reshape(test_labels.shape[0],-1).T

In [5]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [6]:
Train_labels=One_hot_encoder(train_labels,10)
#rint(Train_labels.shape)
Train_labels=Train_labels[0]
print(Train_labels.shape)
#rint(train_labels[0])
#rint(Train_labels[0][0])
print(train_dataset.shape)

(50000, 10)
(784, 50000)


In [7]:
total_layers=7
Layer_Units={1:38,2:38,3:38,4:38,5:38,6:38}
Layer_Units[total_layers]=10
cache={}
for i in range(1,total_layers+1):
    cache['Z'+str(i)]=0
    cache['A'+str(i)]=0
parameters={}
count=1
for i in Layer_Units:
    num=i
    weight="W"+str(num)
    bias='b'+str(num)
    if i==1:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],784],initializer=tf.contrib.layers.xavier_initializer(seed=1))
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())
    else:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],Layer_Units[i-1]],initializer=tf.contrib.layers.xavier_initializer(seed=1)) 
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())

In [8]:
print(train_dataset.shape)
print(Train_labels.shape)

(784, 50000)
(50000, 10)


In [21]:
for i in Layer_Units:
        num=i
        if i==1:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],train_dataset),parameters['b'+str(num)])
            cache['A'+str(i)]=tf.nn.relu(cache['Z'+str(i)])   
        elif i==total_layers:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],cache['A'+str(i-1)]),parameters['b'+str(num)])
        else:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],cache['A'+str(i-1)]),parameters['b'+str(num)])
            cache['A'+str(i)]=tf.nn.relu(cache['Z'+str(i)])

In [22]:
logits = tf.transpose(cache['Z'+str(total_layers)])
labels = Train_labels
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels =labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
count=1
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print('Number','Loss','Accuracy','Wright1')
    for i in range(1500):
        session.run(train_op)
        loss, acc = session.run([cost, accuracy])
        print(count,':',loss,':',acc,':',parameters['W1'][0][0].eval())
        count=count+1   

Number Loss Accuracy Wright1
1 : 2.31139 : 0.14322 : -0.0455896
2 : 2.29728 : 0.14266 : -0.0448873
3 : 2.286 : 0.1859 : -0.0440861
4 : 2.27364 : 0.22874 : -0.0433776
5 : 2.25817 : 0.23156 : -0.0427961
6 : 2.24018 : 0.21254 : -0.0423179
7 : 2.22359 : 0.19388 : -0.0419154
8 : 2.20777 : 0.1882 : -0.0415703
9 : 2.19058 : 0.18888 : -0.0412711
10 : 2.17049 : 0.18898 : -0.0410091
11 : 2.14772 : 0.18886 : -0.0407763
12 : 2.12499 : 0.19904 : -0.0405684
13 : 2.10157 : 0.21518 : -0.0403818
14 : 2.0754 : 0.23216 : -0.0402138
15 : 2.04699 : 0.2386 : -0.0400622
16 : 2.01783 : 0.23906 : -0.0399251
17 : 1.98754 : 0.2477 : -0.0398009
18 : 1.95539 : 0.27464 : -0.0396885
19 : 1.92123 : 0.3191 : -0.0395864
20 : 1.88487 : 0.36214 : -0.0394936
21 : 1.84539 : 0.3963 : -0.0394092
22 : 1.80279 : 0.42162 : -0.0393325
23 : 1.75698 : 0.44972 : -0.0392627
24 : 1.70755 : 0.48684 : -0.039199
25 : 1.6549 : 0.52254 : -0.0391411
26 : 1.59935 : 0.54722 : -0.0390883
27 : 1.54098 : 0.56618 : -0.0390402
28 : 1.48018 : 0.58

222 : 0.15384 : 0.95356 : -0.0385522
223 : 0.155454 : 0.9531 : -0.0385522
224 : 0.157011 : 0.95272 : -0.0385522
225 : 0.154822 : 0.95318 : -0.0385522
226 : 0.151943 : 0.95426 : -0.0385522
227 : 0.150917 : 0.95444 : -0.0385522
228 : 0.151883 : 0.9542 : -0.0385522
229 : 0.152376 : 0.95422 : -0.0385522
230 : 0.150473 : 0.95442 : -0.0385522
231 : 0.148574 : 0.95552 : -0.0385522
232 : 0.148277 : 0.9555 : -0.0385522
233 : 0.148754 : 0.95522 : -0.0385522
234 : 0.148397 : 0.95526 : -0.0385522
235 : 0.146783 : 0.95588 : -0.0385522
236 : 0.145636 : 0.9566 : -0.0385522
237 : 0.145537 : 0.95626 : -0.0385522
238 : 0.14559 : 0.95648 : -0.0385522
239 : 0.145033 : 0.95644 : -0.0385522
240 : 0.143818 : 0.95684 : -0.0385522
241 : 0.142894 : 0.95744 : -0.0385522
242 : 0.142573 : 0.95714 : -0.0385522
243 : 0.142451 : 0.95776 : -0.0385522
244 : 0.142041 : 0.95718 : -0.0385522
245 : 0.141162 : 0.95806 : -0.0385522
246 : 0.140252 : 0.9582 : -0.0385522
247 : 0.139604 : 0.9584 : -0.0385522
248 : 0.139248 : 0.9

438 : 0.0809907 : 0.9769 : -0.0385522
439 : 0.080947 : 0.97628 : -0.0385522
440 : 0.0807175 : 0.9768 : -0.0385522
441 : 0.0805468 : 0.97638 : -0.0385522
442 : 0.0801766 : 0.97702 : -0.0385522
443 : 0.0798484 : 0.97644 : -0.0385522
444 : 0.0794077 : 0.97722 : -0.0385522
445 : 0.0790656 : 0.97684 : -0.0385522
446 : 0.0787382 : 0.97762 : -0.0385522
447 : 0.0784538 : 0.97732 : -0.0385522
448 : 0.0781909 : 0.97758 : -0.0385522
449 : 0.0779634 : 0.97754 : -0.0385522
450 : 0.0777735 : 0.97772 : -0.0385522
451 : 0.0776023 : 0.9778 : -0.0385522
452 : 0.0774439 : 0.97782 : -0.0385522
453 : 0.0772915 : 0.97792 : -0.0385522
454 : 0.0771976 : 0.97752 : -0.0385522
455 : 0.0772003 : 0.97806 : -0.0385522
456 : 0.077396 : 0.97744 : -0.0385522
457 : 0.0779138 : 0.9779 : -0.0385522
458 : 0.0791363 : 0.97656 : -0.0385522
459 : 0.0810995 : 0.97634 : -0.0385522
460 : 0.0855434 : 0.9734 : -0.0385522
461 : 0.0912111 : 0.97232 : -0.0385522
462 : 0.104668 : 0.96556 : -0.0385522
463 : 0.116849 : 0.9625 : -0.0385

651 : 0.0536652 : 0.9845 : -0.0385522
652 : 0.0540755 : 0.98476 : -0.0385522
653 : 0.054548 : 0.9841 : -0.0385522
654 : 0.0539141 : 0.9848 : -0.0385522
655 : 0.0529992 : 0.98486 : -0.0385522
656 : 0.0513392 : 0.98606 : -0.0385522
657 : 0.0502288 : 0.98672 : -0.0385522
658 : 0.0499366 : 0.987 : -0.0385522
659 : 0.0504115 : 0.98682 : -0.0385522
660 : 0.0510658 : 0.9861 : -0.0385522
661 : 0.051147 : 0.9862 : -0.0385522
662 : 0.0506834 : 0.98612 : -0.0385522
663 : 0.0498139 : 0.987 : -0.0385522
664 : 0.0493013 : 0.98718 : -0.0385522
665 : 0.049305 : 0.98704 : -0.0385522
666 : 0.0495799 : 0.98708 : -0.0385522
667 : 0.0496497 : 0.98656 : -0.0385522
668 : 0.0493749 : 0.9871 : -0.0385522
669 : 0.0488539 : 0.98738 : -0.0385522
670 : 0.0484193 : 0.98754 : -0.0385522
671 : 0.0482885 : 0.9875 : -0.0385522
672 : 0.0484231 : 0.98734 : -0.0385522
673 : 0.0485704 : 0.9875 : -0.0385522
674 : 0.0485043 : 0.9871 : -0.0385522
675 : 0.0482531 : 0.9874 : -0.0385522
676 : 0.0479066 : 0.98748 : -0.0385522
677

864 : 0.0465664 : 0.98768 : -0.0385522
865 : 0.0464729 : 0.98776 : -0.0385522
866 : 0.0463814 : 0.98776 : -0.0385522
867 : 0.0462894 : 0.98776 : -0.0385522
868 : 0.0461984 : 0.98774 : -0.0385522
869 : 0.0461086 : 0.98778 : -0.0385522
870 : 0.046019 : 0.9878 : -0.0385522
871 : 0.0459303 : 0.98786 : -0.0385522
872 : 0.0458425 : 0.98792 : -0.0385522
873 : 0.045754 : 0.98794 : -0.0385522
874 : 0.0456685 : 0.988 : -0.0385522
875 : 0.0455831 : 0.98802 : -0.0385522
876 : 0.0454987 : 0.98806 : -0.0385522
877 : 0.0454147 : 0.9881 : -0.0385522
878 : 0.0453311 : 0.9881 : -0.0385522
879 : 0.0452484 : 0.98808 : -0.0385522
880 : 0.0451657 : 0.98818 : -0.0385522
881 : 0.045085 : 0.9882 : -0.0385522
882 : 0.0450041 : 0.98828 : -0.0385522
883 : 0.0449244 : 0.98822 : -0.0385522
884 : 0.0448455 : 0.98822 : -0.0385522
885 : 0.044767 : 0.98828 : -0.0385522
886 : 0.0446896 : 0.9883 : -0.0385522
887 : 0.0446121 : 0.98836 : -0.0385522
888 : 0.0445358 : 0.98834 : -0.0385522
889 : 0.0444598 : 0.98836 : -0.03855

1074 : 0.0340212 : 0.99204 : -0.0385522
1075 : 0.0339768 : 0.99206 : -0.0385522
1076 : 0.0339325 : 0.99208 : -0.0385522
1077 : 0.0338872 : 0.99206 : -0.0385522
1078 : 0.0338434 : 0.99214 : -0.0385522
1079 : 0.0337998 : 0.9921 : -0.0385522
1080 : 0.0337527 : 0.9921 : -0.0385522
1081 : 0.0337084 : 0.99212 : -0.0385522
1082 : 0.0336641 : 0.99218 : -0.0385522
1083 : 0.033619 : 0.99218 : -0.0385522
1084 : 0.0335745 : 0.99216 : -0.0385522
1085 : 0.0335304 : 0.99224 : -0.0385522
1086 : 0.0334862 : 0.9922 : -0.0385522
1087 : 0.0334416 : 0.99222 : -0.0385522
1088 : 0.0333976 : 0.99222 : -0.0385522
1089 : 0.0333532 : 0.99224 : -0.0385522
1090 : 0.0333092 : 0.99224 : -0.0385522
1091 : 0.033265 : 0.99232 : -0.0385522
1092 : 0.033222 : 0.99232 : -0.0385522
1093 : 0.0331789 : 0.99226 : -0.0385522
1094 : 0.0331337 : 0.99232 : -0.0385522
1095 : 0.0330896 : 0.99238 : -0.0385522
1096 : 0.0330453 : 0.99238 : -0.0385522
1097 : 0.033001 : 0.99234 : -0.0385522
1098 : 0.0329572 : 0.99236 : -0.0385522
1099 : 

1281 : 0.0254296 : 0.99468 : -0.0385522
1282 : 0.0253926 : 0.9947 : -0.0385522
1283 : 0.0253535 : 0.99468 : -0.0385522
1284 : 0.0253156 : 0.9947 : -0.0385522
1285 : 0.0252753 : 0.99472 : -0.0385522
1286 : 0.0252366 : 0.9947 : -0.0385522
1287 : 0.0251982 : 0.9947 : -0.0385522
1288 : 0.025159 : 0.9947 : -0.0385522
1289 : 0.0251241 : 0.9947 : -0.0385522
1290 : 0.0250852 : 0.9947 : -0.0385522
1291 : 0.0250506 : 0.99468 : -0.0385522
1292 : 0.0250171 : 0.99474 : -0.0385522
1293 : 0.0249742 : 0.99472 : -0.0385522
1294 : 0.0249319 : 0.9947 : -0.0385522
1295 : 0.0248925 : 0.99476 : -0.0385522
1296 : 0.0248534 : 0.99474 : -0.0385522
1297 : 0.0248168 : 0.99474 : -0.0385522
1298 : 0.0247794 : 0.99476 : -0.0385522
1299 : 0.0247424 : 0.99476 : -0.0385522
1300 : 0.0247089 : 0.99474 : -0.0385522
1301 : 0.0246773 : 0.99478 : -0.0385522
1302 : 0.0246389 : 0.99482 : -0.0385522
1303 : 0.024598 : 0.99482 : -0.0385522
1304 : 0.0245551 : 0.99484 : -0.0385522
1305 : 0.0245171 : 0.99482 : -0.0385522
1306 : 0.0

1488 : 0.0183445 : 0.99676 : -0.0385522
1489 : 0.0183215 : 0.99678 : -0.0385522
1490 : 0.0182827 : 0.99676 : -0.0385522
1491 : 0.0182453 : 0.99676 : -0.0385522
1492 : 0.0182129 : 0.99682 : -0.0385522
1493 : 0.0181837 : 0.99678 : -0.0385522
1494 : 0.0181577 : 0.99678 : -0.0385522
1495 : 0.0181225 : 0.99686 : -0.0385522
1496 : 0.0180855 : 0.99684 : -0.0385522
1497 : 0.0180477 : 0.99684 : -0.0385522
1498 : 0.0180202 : 0.99686 : -0.0385522
1499 : 0.0179911 : 0.99682 : -0.0385522
1500 : 0.0179537 : 0.9969 : -0.0385522
